### Objective

In this notebook, we investigate the Bayesian optimization strategy for PE optimization.

In [1]:
import copy
import pickle
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler 
from scipy.stats import qmc, norm
import gpflow
from sklearn.metrics import brier_score_loss

import utility
from two_sources import thermal_distribution_maxT
from manager import region_manager

### 0. Setup necessary constants

In [2]:
Tjmax = 175

# Bounds: d, b, L, c, L_duct, n
lb = np.array([5e-3, 73.7e-3, 127.2e-3, 10e-3, 20e-3, 10])
ub = np.array([30e-3, 307e-3, 530e-3, 39e-3, 50e-3, 50])
Data = (25, 50e-3, 65e-3, 61.4e-3, 106e-3)

### 1. Load dataset

In [3]:
df_train = pd.read_csv('./dataset/passive_learning_train.csv')
df_train.columns = ['Q1', 'Q2', 'd', 'b', 'L', 'c', 'L_duct', 'n', 't', 'xc1', 'yc1', 'xc2', 'yc2', 'Tc', 'Tj', 'w']
print(f"Training pool: {df_train.shape[0]}")

Training pool: 939


In [4]:
df_candidates = pd.read_csv('./dataset/candidates.csv')
df_candidates.columns = ['d', 'b', 'L', 'c', 'L_duct', 'n', 't', 'xc1', 'yc1', 'xc2', 'yc2']
print(f"Candidate pool: {df_candidates.shape[0]}")

Candidate pool: 470070


In [5]:
# Dedicated testing set
df_test = pd.read_csv('./dataset/test.csv')
df_test.columns = ['Q1', 'Q2', 'd', 'b', 'L', 'c', 'L_duct', 'n', 't', 'xc1', 'yc1', 'xc2', 'yc2', 'Tc', 'Tj', 'w']
print(f"Testing dataset: {df_test.shape[0]}")

Testing dataset: 5655


In [7]:
# Extract train data
X, _, y, _ = utility.create_samples(df_train, len(df_train))
X_test, _, y_test, _ = utility.create_samples(df_test, len(df_test))

# Normalization
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)

### 2. BO iterations

#### 2.1 Initial training

In [9]:
%%time

X_train = copy.deepcopy(X)
X_train_scaled = copy.deepcopy(X_scaled)
y_train = copy.deepcopy(y)

GP = utility.fit(X_train_scaled, y_train, n_restarts=10, trainable=True, verbose=True)

# Load GP model
# with open('./model/PL_model_params_939.pickle', 'rb') as handle:
#     GP_params = pickle.load(handle)
# GP = utility.load_GP_model(GP_params, X, y)

model = copy.deepcopy(GP)
init_lengthscales = model.kernel.lengthscales.numpy().reshape(1, -1)
init_variance = model.kernel.variance.numpy().flatten()[0]

Performing 1-th optimization:
Performing 2-th optimization:
Performing 3-th optimization:
Performing 4-th optimization:
Performing 5-th optimization:
Performing 6-th optimization:
Performing 7-th optimization:
Performing 8-th optimization:
Performing 9-th optimization:
Performing 10-th optimization:
CPU times: total: 19min 41s
Wall time: 4min 21s


In [10]:
X_test_norm = scaler.transform(X_test)
f_mean, f_var = model.predict_f(X_test_norm, full_cov=False)
y_prob = norm.cdf(175, loc=f_mean, scale=np.sqrt(f_var))
label = np.where(y_test > 175, 1, 0)
brier_score = brier_score_loss(label, 1-y_prob)
    
rmse, max_e, max_per, _, mean_per = utility.evaluate_model(y_test, f_mean.numpy().flatten())
print(f"RMSE: {rmse:.4f} / data std: {np.std(y_test):.4f}")
print(f"Max Error: {max_e:.4f}")
print(f"Max Percentage Error: {max_per:.2f}")
print(f"Mean Percentage Error: {mean_per:.2f}")
print(f"Brier score: {brier_score:.5f}")

RMSE: 8.7686 / data std: 32.8753
Max Error: 99.1912
Max Percentage Error: 40.15
Mean Percentage Error: 4.92
Brier score: 0.01932


#### 2.2 Given Q1-Q2 specification

In [ ]:
# Compose dataset
Q1, Q2 = 300, 200
Q1_array, Q2_array = Q1*np.ones((df_candidates.shape[0], 1)), Q2*np.ones((df_candidates.shape[0], 1))
X_candidates = df_candidates.to_numpy()
X_candidates = np.hstack((Q1_array, Q2_array, X_candidates))

In [ ]:
%%time
f_mean, f_var = utility.GP_predict_candidates(model, X_candidates, scaler)

**Note that indeed the inference time highly depends on the training data size.**

939 ==> 48.3s

200 ==> 5.14s

#### 2.3 BO setup

In [ ]:
%%time

# 1-Select promising candidates from the pool
n_batch = 10
acq, indices = utility.acquisition(None, f_mean, f_var, X_candidates, scaler, 
                                   Tjmax, batch_mode=True, batch_size=n_batch)
print("Candidates selected!")

managers = []
for index in indices:

    # Enrich dataset
    X_train = np.vstack((X_train, X_candidates[index]))
    y, _ = thermal_distribution_maxT(X_candidates[index], Data)
    y_train = np.append(y_train, np.array(y))

    # Create managers
    manager = region_manager(model, n_batch, Data, Tjmax, X_candidates[index], Q1, Q2)
    managers.append(manager)

    # Update weight
    if y <= manager.Tjmax:
        w = utility.evaluate_weight(X_candidates[index].reshape(1, -1))[0]
    else:
        w = np.inf
        
    manager.init_weight(w)

print("Managers created!")

#### 2.4 Branch & Bound algorithm

In [ ]:
%%time

counter = 0
status_list = [False] * n_batch
design_list, weight_list = [], []

while np.sum(status_list) < n_batch:
    counter += 1
    print(f"Start {counter}-th iteration:")

    # Update global GP model
    X_train_scaled = scaler.transform(X_train)
    model = utility.fit(X_train_scaled, y_train, n_restarts=1, init_lengthscales=init_lengthscales.flatten(), 
                        init_variance=init_variance, trainable=False, verbose=False)
    print("Global model updated!")
    
    for i, (manager, index) in enumerate(zip(managers, indices)):
        print(f"Process {i+1}/{n_batch} manager:")
        print("Updating manager model:")
        manager.update_model(model)
        
        if not manager.converge_flag:
            X, y = manager.branch_and_bound(lb, ub, scaler, candidate_num=5000)
            X_train = np.vstack((X_train, X))
            y_train = np.append(y_train, np.array(y))
    
            # 5-Check conditions
            design, weight, converge_flag = manager.check_converge()   
            status_list[i] = converge_flag

            if design is not None:
                design_list.append(design)
                weight_list.append(weight)
                print(f"{i+1}/{n_batch} manager done!")
        else:
            print(f"Skip {i+1}/{n_batch} manager.")

print(f"All managers done!")

In [ ]:
# Compile results
best_weight = np.min(weight_list)
best_design = design_list[np.argmin(weight_list)]
Tmax, _ = thermal_distribution_maxT(best_design, Data)
print(f"Best weight: {best_weight}")
print(f"Tmax: {Tmax}")

columns = ['Q1', 'Q2', 'd', 'b', 'L', 'c', 'L_duct', 'n', 't', 'xc1', 'yc1', 'xc2', 'yc2']
df = pd.DataFrame({'optimized': best_design})
df.index = columns
df.transpose()